# Máltækni - Náttúruvernd

# Data import & manipulation

In [10]:
from numpy.random import default_rng
import re
import tqdm
import pandas as pd
import pickle
import numpy as np

In [11]:
#@Vésteinn
def read_news_tsv(file_name):
    data = []
    with open(file_name, encoding="utf-8") as fh:
        for line in tqdm.tqdm(fh.readlines()):
            title, author, url, timestamp, _, _, text, text_tokenized = line.split("\t")
            article = {
                "title": title,
                "author": author,
                "timestamp": timestamp,
                "text": text,
                "text_tokenized": text_tokenized
            }
            data.append(article)
        return data

In [12]:
articles = read_news_tsv("icelandic_news_sample_22480.tsv")
print(f"Article count {len(articles)}")

100%|██████████| 22248/22248 [00:00<00:00, 166342.61it/s]


Article count 22248


In [13]:
articles_df = pd.DataFrame(articles)

In [14]:
articles_Big = read_news_tsv("articles_until_01-11-2020_detok.tsv")
print(f"Article count {len(articles_Big)}")

100%|██████████| 640136/640136 [00:17<00:00, 35898.29it/s]


Article count 640136


In [15]:
news_Big = pd.DataFrame(articles_Big)

In [16]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(news_Big, test_size = 0.15, random_state=11)

In [17]:
print(f'Training data examles: {data_train.shape[0]}')
print(f'Test data examles: {data_test.shape[0]}')

Training data examles: 544115
Test data examles: 96021


In [18]:
data_test, data_test_labeled = train_test_split(data_test, test_size = 0.005, random_state=11)

In [19]:
print(f'Test data examles: {data_test.shape[0]}')
print(f'Labeled test data examles: {data_test_labeled.shape[0]}')

Test data examles: 95540
Labeled test data examles: 481


In [20]:
# Subset af training til að koma virkni í gang án þjálfunartíma
_, data_playground = train_test_split(data_train, test_size = 0.10, random_state=11)
print(f'Playground data examles: {data_playground.shape[0]}')

Playground data examles: 54412


# Skriftupælingar

In [21]:
# Skoðar 'count' margar handahófskenndar fréttir og skilar lista af
# algildum indexum fyrir þær fréttir sem innihalda 'náttúruvernd', 'loftslag' eða 'umhverfi'
def rand_frett(safn, count):
    listi = []
    from numpy.random import default_rng
    import re
    rng = default_rng()
    n = safn.shape[0]
    for i in tqdm.trange(count):
        x = rng.integers(n)
        if pd.isnull(safn.iloc[x]["text"]):
            continue
        frett = safn.iloc[x]["text"] # ath iloc
        frettnr = safn.iloc[x].name
        lykilord = ["náttúruvernd", "loftslag", "umhverfi"]
        fjoldi = 0
        for wrd in lykilord:
            fjoldi += len(re.findall(wrd, frett))
        if fjoldi > 0:
            listi.append(frettnr) # Ath. algildur index
            #print(frett)
    return listi

In [22]:
# Skoðar allar fréttir í gefnu safni og skilar lista með algildum indexum 
# á þeim fréttum sem innihalda fleiri en 'threshold' fjölda af 'náttúruvernd', 'loftslag' eða 'umhverfi'
def allar_frettir(safn, threshold=0):
    listi = []

    n = safn.shape[0]
    for i in tqdm.trange(n):
        frett = safn.iloc[i]["text"]
        lykilord = ["náttúruvernd", "loftslag", "umhverfi"]
        fjoldi = 0
        for wrd in lykilord:
            fjoldi += len(re.findall(wrd, frett))
        if fjoldi > threshold:
            listi.append(i) # ath. afstæð númer m.v. safn
            #print(frett)
    return listi 

In [23]:
# Athuga hvaða hlutfall kemur út úr síunni
sample_count = 100000
big_list = rand_frett(news_Big, sample_count)
print(len(big_list)/sample_count)

100%|██████████| 100000/100000 [00:26<00:00, 3791.55it/s]

0.06697


## Merkjum data_test_labeled

In [24]:
sigtaðar_frettir = allar_frettir(data_test_labeled)

100%|██████████| 481/481 [00:00<00:00, 8975.74it/s]


In [25]:
for fr in sigtaðar_frettir[:3]:
    print(data_test_labeled.iloc[fr])

title             Fyrst var hún hötuð en í dag er hún daglegt brauð
author                                       Ritstjórn Hringbrautar
timestamp                                       2019-04-28 21:09:00
text              (Á mynd: Áður en p-pillan og aðrar getnaðarvar...
text_tokenized    ( Á mynd : Áður en p-pillan og aðrar getnaðarv...
Name: 605530, dtype: object
title                                 „Sá glæsilegasti frá upphafi“
author                                             Ritstjórn mbl.is
timestamp                                       2020-09-05 09:36:00
text              „Veturinn fram undan verður sá þéttasti og glæ...
text_tokenized    „ Veturinn fram undan verður sá þéttasti og gl...
Name: 396081, dtype: object
title                 Tekur við kyndlinum sem tenging við umheiminn
author                                        Bergsteinn Sigurðsson
timestamp                                       2017-04-19 20:15:41
text              Hús sem gert er í nafni Vigdísar Finnbogad

In [26]:
len(sigtaðar_frettir)

38

In [27]:
# for i in sigtaðar_frettir[:3]:
#     frett = data_test_labeled.iloc[i]
#     print(frett["text"])
#     print(frett["title"])
#     einkunn = input("Um náttúruvernd?\n")
#     if einkunn == str(1):
#         ja.append(frett.name)
#     elif einkunn == str(2):
#         vafi.append(frett.name)
#     else:
#         annad.append(frett.name)

In [28]:
# Niðurstöður
jakvætt = {165133,322727,343386,233961,500609,327615}

# !! ATH breyta hér, flokka bara þau sem eru handyfirfarin sem neikvæð 
neikvætt = set(data_test_labeled.index).difference(jakvætt)

print(len(neikvætt))
print(len(jakvætt))
print(data_test_labeled.shape[0])

# data_test_labeled["label"] = np.zeros((len(data_test_labeled), 1), dtype = np.int8)
# for i in jakvætt:
#     data_test_labeled.loc[i,"label"] = 1
# for fr in big_list[:10]:
#     print(news_Big.loc[fr])

475
6
481


## Merkjum data_train

In [29]:
# 'vistaðar niðurstöður' úr flokkun að neðan.
# jákvæðar fréttir úr train
ja =    {285192, 531728, 543888, 290583, 624031, 620326, 586792, 300335, 615601, 57271, 541114, 232512, 203202, 565829, 402387, 212309, 563685, 207852, 319984, 338039, 551033}
# fréttir sem eru kannski já, kannski ekki
vafi =  {188673, 253189, 582670, 549263, 580500, 196829, 138785, 201385, 146412, 624949, 467836, 18621}
# Neikvæðar fréttir sem komu úr síunni
annad = {425218, 398339, 232836, 377475, 40966, 108040, 61449, 411913, 441611, 188049, 594451, 357913, 389534, 447519, 403616, 598944, 403230, 438562, 383270, 300455, 260392, 545578, 245419, 496045, 280365, 111155, 77492, 616884, 26553, 598717, 397246, 264765, 180800, 353226, 292173, 217422, 53844, 179030, 306134, 524889, 622682, 288093, 557662, 146272, 471780, 628965, 111590, 180717, 506996, 129525, 423028, 19063, 409848, 290171}
# # Neikvæðar fréttir teknar af handahófi úr train
neinei = {41758, 45206, 94336, 122610, 133607, 152981, 190982, 210392, 235700, 258454, 276324, 294168, 303348, 328102, 333446, 344739, 351967, 356145, 376980, 426847, 429708, 437621, 460035, 461629, 524825, 566916, 581492, 612748, 631359, 631908}
possibly_corrupt = {628965, 409848, 549263}
print(ja, vafi, annad, sep="\n")
print(len(ja))

{285192, 531728, 543888, 290583, 624031, 620326, 586792, 300335, 615601, 57271, 541114, 232512, 203202, 565829, 402387, 212309, 563685, 207852, 319984, 338039, 551033}
{188673, 138785, 253189, 201385, 146412, 582670, 549263, 580500, 624949, 18621, 467836, 196829}
{425218, 398339, 232836, 377475, 40966, 108040, 61449, 411913, 441611, 188049, 594451, 357913, 403230, 389534, 403616, 598944, 438562, 447519, 383270, 300455, 260392, 545578, 245419, 496045, 280365, 111155, 77492, 616884, 26553, 598717, 397246, 264765, 180800, 353226, 292173, 217422, 53844, 179030, 306134, 524889, 622682, 288093, 557662, 146272, 471780, 628965, 111590, 180717, 506996, 129525, 423028, 19063, 409848, 290171}
21


In [30]:
def label(safn, sigti, count=-1):
    for nr, i in enumerate(sigti[:count]):
        if i in ja or i in vafi or i in annad:
            continue
        frett = safn.loc[i] # Gert er ráð fyrir global index
        print(f"Frétt nr. {nr}")
        print(frett["text"])
        print(frett["title"])
        einkunn = input("Um náttúruvernd?\n")
        print()
        if einkunn == str(1):
            ja.add(frett.name)
        elif einkunn == str(2):
            vafi.add(frett.name)
        elif einkunn == str(0):
            neinei.add(frett.name)
        else:
            annad.add(frett.name)

In [31]:
# label(data_train, data_train.index)

Sigta af handahófi úr train <br>
Flokka sigtað í já og nei<br>
flokka af handahófi úr ósigtuðu (svo nei sé ekki bara sigtað nei)<br>
    - kannski í sér hóp til að sjá hvort það endurspeglist í flokkunum?<br>
Þegar það eru komnir nokkrir tugir í já, þá athuga hvar flokkaða lendir í LDA<br>
Kannski finna gögn sem eru sérstaklega um náttúruvernd, fréttir frá samtökum, Sólin, ...<br>

In [32]:
slembsigti = rand_frett(data_train, 2000) # Hefði kannski átt að setja seed?

100%|██████████| 2000/2000 [00:00<00:00, 3672.20it/s]


In [33]:
# label(data_train, slembsigti)

In [34]:
playground_sigti = allar_frettir(data_playground, 5)

100%|██████████| 54412/54412 [00:04<00:00, 11434.34it/s]


In [35]:
len(playground_sigti)

229

# GENSIM LDA pælingar

In [36]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [37]:
import string
stop_words = set()
extra_stop_words = ["var", "er", "ekki", "sé"]
with open("is_stop_words.txt") as stop_word_file:
    for line in stop_word_file.readlines():
        stop_words.add(line.strip())
stop_words.update(set(string.punctuation))
stop_words.update(set(extra_stop_words))

In [38]:
import tokenizer
from islenska import Bin

b = Bin()

def naive_bin_lemma(token):
    candidates = b.lookup_lemmas_and_cats(token)
    if not candidates:
        return None
    if len(candidates) > 1:
        return None
    cand = candidates.pop()
    if cand[1] not in ["kk", "kvk", "hk"]:
        return None
    return cand[0]


def tokenize_and_lemmatize(sentence):
    if pd.isnull(sentence): # Líklegast óþarft núna
            return []
    tokenized_text = list(tokenizer.split_into_sentences(sentence))
    lemmas = []
    for sent in tokenized_text:
        for token in sent.split():
            if token in stop_words:
                continue
            lemma = naive_bin_lemma(token.lower())
            if lemma is not None: # bæta við stop_words hér? and lemma not in stop_words
                lemmas.append(lemma)
    return lemmas

In [39]:
def load_all():
    with open('data_words.pickle', 'rb') as fh:
        data_words = pickle.load(fh)
    with open('data_words_bigrams.pickle', 'rb') as fh:
        data_words_bigrams = pickle.load(fh)
    with open('id2word.pickle', 'rb') as fh:
        id2word = pickle.load(fh)
    with open('corpus.pickle', 'rb') as fh:
        corpus = pickle.load(fh)
    lda_model = gensim.models.ldamulticore.LdaModel.load("Big_lda_model")
    
    return data_words, data_words_bigrams, id2word, corpus, lda_model

In [40]:
def build_all():
    data_words = [tokenize_and_lemmatize(a) for a in tqdm.notebook.tqdm(data_train["text_tokenized"])]

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    # See trigram example
    # print(trigram_mod[bigram_mod[data_words[0]]])
    data_words_bigrams = make_bigrams(tqdm.tqdm(data_words))
    # Create Dictionary
    id2word = corpora.Dictionary(data_words)

    # Create Corpus
    texts = data_words

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=10, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)

In [41]:
def save_all():
    with open('data_words.pickle', 'wb') as fh:
        pickle.dump(data_words, fh)
    with open('data_words_bigrams.pickle', 'wb') as fh:
        pickle.dump(data_words_bigrams, fh)
    with open('id2word.pickle', 'wb') as fh:
        pickle.dump(id2word, fh)
    with open('corpus.pickle', 'wb') as fh:
        pickle.dump(corpus, fh)
    lda_model.save("./Big_lda_model")

In [43]:
data_words, data_words_bigrams, id2word, corpus, lda_model = load_all()

UnpicklingError: invalid load key, 'v'.

## Tilraunir

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Sjáum orðin tengd ákveðnum flokk
[id2word[i[0]] for i in lda_model.get_topic_terms(6, topn=50)]

In [ ]:
# Fáum hóp ákveðins orðs í safninu
# Af hverju sýnir þetta fyrir fæst orð?
lda_model.get_term_topics(id2word.token2id["króna"])

In [ ]:
sport = """„Úrslitin eru það mikilvægasta sem við tökum með okkur úr leik kvöldsins. Það er afrek að vinna Porto á útivelli. Að sigra eins og við sigruðum gerir sigurinn enn sætari,“ sagði Jürgen Klopp að loknum 5-1 sigri Liverpool á Drekavöllum í kvöld.
„Það var mikið af góðum augnablikum í leik kvöldsins. Við gátum séð að Porto horfði á leikinn okkar gegn Brentford þar sem þeir voru mjög beinskeyttir, ég vildi lagfæra það á vellinum og við gerðum það skref fyrir skref.“
"""
sport = tokenize_and_lemmatize(sport)

In [ ]:
covid= """Í gær greindust 36 einstaklingar innanlands með Covid-19 og voru 28 í sóttkví við greiningu. Tuttugu voru óbólusettir. 348 einstaklingar eru í einangrun hér á landi vegna sjúkdómsins og 1.164 í sóttkví. """
covid=tokenize_and_lemmatize(covid)

In [ ]:
def classify(text, verbose=False):
    bow_vector = id2word.doc2bow(text)
    #print(lda_model[bow_vector])
    for index, score in sorted(lda_model[bow_vector][0], key=lambda tup: -1*tup[1]):
        if verbose:
            print("Cluster: {}\t Score: {}\t Topic: {}".format(index, score, lda_model.print_topic(index, 5)))
    if verbose:
        print()
    return(lda_model[bow_vector][0])

In [53]:
def lda_group(indexes, verbose=False):
    mat = np.zeros((len(indexes), 10), dtype=np.float32)
    for i, example in enumerate(tqdm.tqdm(indexes)):
        text = data_train.loc[example]["text_tokenized"] # getur verið news_Big fyrir öll dæmin
        lemm_text = tokenize_and_lemmatize(text)
        vec = classify(lemm_text)
        for group, score in vec:
            mat[i, group] = score
    score_sum = np.sum(mat, axis=0)
    if verbose:
        print(score_sum)
        prin3t(score_sum/sum(score_sum))
        print(-np.sort(-score_sum))
        print(np.argsort(-score_sum))
    return mat

In [54]:
jalist = list(ja)

In [100]:
ja_mat = lda_group(jalist, False)

  0%|          | 0/21 [00:00<?, ?it/s]


AttributeError: 'RobertaTokenizerFast' object has no attribute 'split_into_sentences'

In [50]:
np.argmax(ja_mat, axis=1)

NameError: name 'ja_mat' is not defined

In [51]:
jalist[0]

285192

In [ ]:
lda_group([285192])

In [ ]:
data_train.loc[285192]

In [ ]:
# atta = [3,6,19,20]
# þristur = [8,12,15,16]
# sexa = [1,2,4,5,7,8,9,10,11,13,14,17]
# 
# for i in atta:
#     print(data_train.loc[jalist[i]]["title"])
# print()
# for i in þristur:
#     print(data_train.loc[jalist[i]]["title"])
# print()
# for i in sexa:
#     print(data_train.loc[jalist[i]]["title"])

In [52]:
neineilist = list(neinei)
nei_mat = lda_group(neineilist)
np.argmax(nei_mat, axis=1)

NameError: name 'lda_group' is not defined

In [ ]:
data_train.loc[neineilist[2]]

In [ ]:
import matplotlib.pyplot as plt
def visual_groups(vigur):
    plt.bar(list(range(10)), vigur)
    plt.show()
    
    plt.bar(range(len(vigur)), -np.sort(-vigur))
    plt.xticks(range(len(vigur)), np.argsort(-vigur))
    plt.show()
    
    # Athuga að sýna líka topphópa ekki bara summu einkunna

In [ ]:
visual_groups(np.sum(lda_group(annad, False), axis=0))

In [ ]:
# Keyrir í tæpar 40 mín, stallar á 95% að því er virðist að eilífu
# playground_groups = lda_group(data_playground.index, True)

In [ ]:
# Pælingar á ís.

# for j in data_test_labeled[data_test_labeled["label"] == 1]["text"]:
#     classify(tokenize_and_lemmatize(j))
#     print(j, "\n")
# 
# data_test_labeled[data_test_labeled["label"] == 0][:10]["text"]
# for nei in data_test_labeled[data_test_labeled["label"] == 0][:10]["text"]:
#     classify(tokenize_and_lemmatize(nei))
#     print(nei, "\n")
# for nei in data_test_labeled[data_test_labeled["label"] == 0][:10]:
#     print(nei)
#     texti = nei["text"]
#     print(nei["title"])
#     classify(tokenize_and_lemmatize(texti))
#     print(texti, "\n")

# TF IDF pælingar

In [44]:
from gensim import models
from gensim import matutils
import time

In [ ]:
tfidf = models.TfidfModel(corpus) # sami corpus og fyrir LDA
corpus_tfidf = tfidf[corpus] # lazy

In [ ]:
lsi_200 = models.LsiModel.load("lsi_model_200")
corpus_lsi_200 = lsi_200[corpus_tfidf] 

In [ ]:
lsi_500 = models.LsiModel.load("lsi_model_500")
corpus_lsi_500 = lsi_500[corpus_tfidf] 

In [ ]:
load = True
if not load:
    t = time.time()
    lsi_model = models.LsiModel(corpus_tfidf, id2word=id2word, num_topics=200)  
    corpus_lsi = lsi_model[corpus_tfidf] 
    print(time.time() - t)

In [ ]:
data_train.loc[jalist[2]]["text"]
print(data_words[data_train.index.get_loc(2826)])


In [ ]:
topic_vector = ["friðland", "andrúmsloft", "umhverfissjónarmið", "loftslagsáhrif", "sjávarborð", "loftslagsvísindi", "ísöld", "náttúruhamfarir", "loftslagslíkan", "friðun", "loftslag-breyting", "umhverfisverndarsinni", "losun", "heildarlosun", "kol", "umhverfisverndarstofa", "umhverfisdómstóll", "auðlindaráðuneyti", "loftslagsstefna", "orkunotkun", "umhverfisskattur", "aðgerðasinni", "hamfarahlýnun", "skref", "útlosun", "lifnaðarháttur", "náttúruauðlind", "náttúruverndarlög", "umhverfisfræði", "orkugjafi", "loftslagsmál", "landnotkun", "úrgangur", "sóun", "útblástur", "gráðurhúsaáhrif", "stóriðja", "kolefnishlutleysi", "umhverfisþing", "umhverfisáhrif", "loftslagsbreyting", "súrnun", "sjór", "vistkerfi", "hringrásarhagkerfi", "sjálfbærni", "parísarsamkomulag", "gróðurhúsalofttegund", "loftslagsaðgerðir", "loftslagsvá", "náttúruvernd", "loftslag", "umhverfi", "náttúra", "hlýnun", "jörð", "loftslagsráðstefna", "þjóðgarður", "bráðna", "jökull", "umhverfisvernd", "mengun", "jarðefnaeldsneyti", "náttúruverðmæti"]
topic_vector.sort()
t = lsi_500[tfidf[id2word.doc2bow(topic_vector)]]
a = corpus_lsi_500[300]
# sama og 
# a = lsi_model[tfidf[corpus[300]]]
matutils.cossim(a, t)

In [ ]:
lsi_200.print_topics(10)

In [ ]:
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#logging.disable()


In [ ]:
#for doc, as_text in zip(corpus_lsi, data_train["text"][:55000]):
#    print(doc, as_text)
#

In [ ]:
maxs, maxi = (-1,-1)
for i in range(10000):
    a = lsi_200[tfidf[corpus[i]]]
    sim = matutils.cossim(a,t)
    if sim > maxs:
        maxs = sim
        maxi = i
print(maxs, maxi)

In [ ]:
topic_vector

In [ ]:
for i in range(10000):
    a = lsi_200[tfidf[corpus[i]]]
    sim = matutils.cossim(a,t)
    if sim > 0.15:
        print(data_train.iloc[i]["title"])



In [ ]:
for i in jalist:
    a = corpus_lsi_200[data_train.index.get_loc(i)]
    sim = matutils.cossim(a,t)
    print(sim, data_train.loc[i]["title"])

In [ ]:
data_train.loc[jalist[8]]["title"]

In [ ]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=12)


In [ ]:
query_document = topic_vector


In [ ]:
query_bow = id2word.doc2bow(query_document)


In [ ]:
# Krassar
# sims = index[tfidf[query_bow]]
# index.get_similarities(tfidf[id2word.doc2bow(topic_vector)])


In [ ]:
print(list(enumerate(sims)))

# NER

In [45]:
from transformers import AutoModelForTokenClassification
from transformers import pipeline
from transformers import AutoTokenizer
    
model = AutoModelForTokenClassification.from_pretrained("eliasbe/IceBERT-finetuned-ner")
tokenizer = AutoTokenizer.from_pretrained("eliasbe/IceBERT-finetuned-ner")
iceb = pipeline("ner", model=model, tokenizer=tokenizer)

In [46]:
modelX = AutoModelForTokenClassification.from_pretrained("eliasbe/XLMR-ENIS-finetuned-ner")
# Picked an arbitrary model_max_length to suppress truncation error, 
# not sure if this introduces other problems - KJP
tokenizerX = AutoTokenizer.from_pretrained("eliasbe/XLMR-ENIS-finetuned-ner", model_max_length=512)
xlmr = pipeline("ner", model=modelX, tokenizer=tokenizerX)

Það er hámarkslengd á inputi í þessi módel, þarf að finna lausn á því

In [88]:
xlmr(data_train.loc[jalist[1]]["text_tokenized"])

[{'entity': 'B-Person',
  'score': 0.9986474,
  'index': 2,
  'word': '▁Donald',
  'start': 11,
  'end': 18},
 {'entity': 'B-Person',
  'score': 0.99856585,
  'index': 3,
  'word': 's',
  'start': 18,
  'end': 19},
 {'entity': 'I-Person',
  'score': 0.9980811,
  'index': 4,
  'word': '▁Trump',
  'start': 19,
  'end': 25},
 {'entity': 'B-Location',
  'score': 0.9972326,
  'index': 6,
  'word': '▁Bandaríkjunum',
  'start': 27,
  'end': 41},
 {'entity': 'B-Person',
  'score': 0.9985484,
  'index': 27,
  'word': '▁Bar',
  'start': 122,
  'end': 126},
 {'entity': 'B-Person',
  'score': 0.9984901,
  'index': 28,
  'word': 'acks',
  'start': 126,
  'end': 130},
 {'entity': 'I-Person',
  'score': 0.9974628,
  'index': 29,
  'word': '▁Obama',
  'start': 130,
  'end': 136},
 {'entity': 'B-Person',
  'score': 0.99874073,
  'index': 54,
  'word': '▁Johns',
  'start': 245,
  'end': 251},
 {'entity': 'I-Person',
  'score': 0.998309,
  'index': 55,
  'word': '▁Hold',
  'start': 251,
  'end': 256},
 {

In [57]:

# Athuga að við viljum væntanlega halda utan um staðsetningu líka
def get_names(text):
    l = list()
    end = -1
    beg = False
    for entry in xlmr(text):
        if entry['entity'] in {'B-Person', 'I-Person'}:                               
            word = entry['word'].strip() 
            if word in set(string.punctuation):
                end = int(entry['end'])
                continue
            if word[0] == '▁':
                word = word[1:]
                beg = True
            if end == int(entry['start']):
                last = l[-1]
                if beg:
                    nxt = " ".join([last, word])
                else:
                    nxt = "".join([last, word])
                l = l[:-1]
                l.append(nxt)
            else:
                l.append(word)
            end = int(entry['end'])
        beg = False
    l = set(l)
    return l
            

In [117]:
get_names(alþingi[0])

{'Adda María Jóhannsdóttir'}

In [110]:
# Convert a single name to nefnifall
def nefnifall_stk(nafn):
    out = list()
    temp = list()
    cats = list()
    nöfn = nafn.split()

    # Athuga ef nöfnin hafa mörg kyn, kjósa um það
    for n in nöfn:
        kyn = list(b.lookup_cats(n))
        for k in kyn:
            cats.append(k)
    if len(set(cats)) > 1:
        k = max(set(cats), key=cats.count)
    elif cats:
        k = cats[0]
    #print(k)

    # Setja nefnifallið í lista
    for n in nöfn:   
        try: # Fyrir erlend nöfn
            variant = b.lookup_variants(n, k, ("NF", "ET"))
        except:
            variant = None
        #print(variant)
        if variant: 
            temp.append(variant[0].bmynd)
        else:
            temp.append(n)
        name_nf =" ".join(temp)
    return name_nf

# Convert a list of names to nf.
def nefnifall(nafnalisti):
    out = list()
    for nafn in nafnalisti:
        out.append(nefnifall_stk(nafn))
    return out

In [111]:
nefnifall(get_names(data_train.loc[jalist[3]]["text_tokenized"]))

['Jón Helgi Björnsson', 'Jón', 'Jón Helgi']

In [118]:
alþingi_sample = ["Öddu Maríu Jóhannsdóttur. Varaþingmaður Suðvesturkjördæmis febrúar 2018 (Samfylkingin).", "Adda Bára Sigfúsdóttir. Varaþingmaður Reykvíkinga nóvember 1957 (Alþýðubandalag).", "Adolf H. Berndsen. Alþingismaður Norðurlands vestra 2003 (Sjálfstæðisflokkur)."]
for sentence in alþingi_sample:
    print(get_names(sentence))
    print(nefnifall(get_names(sentence)))

{'Öddu Maríu Jóhannsdóttur'}
['Adda María Jóhannsdóttir']
{'Adda Bára Sigfúsdóttir'}
['Adda Bára Sigfúsdóttir']
{'Adolf H Berndsen'}
['Adolf H Berndsen']


In [65]:
flokkamap = {'Alþýðubandalag': 'Alþýðubandalagið',
 'Alþýðubandalagið': 'Alþýðubandalagið',
 'Alþýðuflokkur': 'Alþýðuflokkurinn',
 'Alþýðuflokkurinn': 'Alþýðuflokkurinn',
 'Borgaraflokkur': 'Borgaraflokkurinn',
 'Borgaraflokkurinn': 'Borgaraflokkurinn',
 'Borgarahreyfingin': 'Borgarahreyfingin',
 'Bændaflokkur': 'Bændaflokkurinn',
 'Bændaflokkurinn': 'Bændaflokkurinn',
 'Framfaraflokkurinn': 'Framfaraflokkurinn',
 'Framsóknar': 'Framsóknarflokkurinn',
 'Framsóknarflokkur': 'Framsóknarflokkurinn',
 'Framsóknarflokkurinn': 'Framsóknarflokkurinn',
 'Frjálslyndi': 'Frjálslyndi',
 'Heimastjórnarflokkur': 'Heimastjórnarflokkurinn',
 'Heimastjórnarflokkurinn': 'Heimastjórnarflokkurinn',
 'Kommúnistaflokkurinn': 'Kommúnistaflokkurinn',
 'Landvarnarflokkurinn': 'Landvarnarflokkurinn',
 'Miðflokkurinn': 'Miðflokkurinn',
 'Píratar': 'Píratar',
 'Sambandsflokkurinn': 'Sambandsflokkurinn',
 'Sameiningarflokkur': 'Sameiningarflokkur',
 'Samfylkingin': 'Samfylkingin',
 'Sjálfstæðisflokkur': 'Sjálfstæðisflokkurinn',
 'Sjálfstæðisflokkurinn': 'Sjálfstæðisflokkurinn',
 'Sparnaðarbandalagið': 'Sparnaðarbandalagið',
 'Sósíalistaflokkur': 'Sósíalistaflokkurinn',
 'Sósíalistaflokkurinn': 'Sósíalistaflokkurinn',
 'Utanflokkabandalagið': 'Utanflokkabandalagið',
 'Vinstrihreyfingin': 'VG',
 'Viðreisn': 'Viðreisn',
 'Íhaldsflokkurinn': 'Íhaldsflokkurinn',
 'Þjóðræðisflokkurinn': 'Þjóðræðisflokkurinn',
 'Þjóðvaki': 'Þjóðvaki',
 'Þjóðvarnarflokkurinn': 'Þjóðvarnarflokkurinn',
 'Kvennalistinn': 'Kvennalistinn',
 'vennalistinn': 'Kvennalistinn',
 'reisn': 'Viðreisn',
 'Björt framtíð': 'Björt framtíð', 
 'Björt': 'Björt framtíð',
 'Samtök um kvennalista': 'Samtök um kvennalista',
 'Samtök um jafnrétti og félagshyggju': 'Samtök um jafnrétti og félagshyggju'
}

In [89]:
def get_party(text):
    l = list()
    end = -1
    beg = False
    for entry in xlmr(text):
        if entry['entity'] == 'B-Organization':                               
            word = entry['word']
            if word in set(string.punctuation):
                end = int(entry['end'])
                continue
            if word[0] == '▁':
                word = word[1:]
                beg = True
            if word[0] in set(string.punctuation):
                word = word[1:]
            if end == int(entry['start']):
                last = l[-1]
                if beg:
                    nxt = " ".join([last, word])
                else:
                    nxt = "".join([last, word])
                l = l[:-1]
                l.append(nxt)
            else:
                l.append(word) # Athuga með að sía fyrir raunverulegum flokkum
            end = int(entry['end'])
        beg = False
    out = set()
    for flokk in l:
        if flokk in flokkamap:
            out.add(flokkamap[flokk])
        else:
            print("Party not found: ", flokk)
    
    # Bara fyrir alþingismannatalið
    if not out:
        party = re.findall("\(((?:\w|\s)*)\)\.", text)
        for p in party:
            if p in flokkamap:
                out.add(p)
            else:
                print(p)
    return out
            

In [86]:
# https://www.althingi.is/altext/cv/is/
with open("Alþingismenn.txt", "r") as f:
    alþingi = f.readlines()

member_party = dict()
for sentence in alþingi:
    
    # ML methods
    member = get_names(sentence)
    party = get_party(sentence)
    
    # RE methods
    # search = re.search("^(.*)\. ", sentence)
    # member = search[0]
    # party = re.finditer("\((.*)\)\.", sentence)
    
    #print(member, party)
    for m in member:       
        member_party[m] = party

Reyk
inga
Samtök
ram
flokkur
Samtök
Reyk
inga
Samtök
Siglfirðinga
varaþingmaður
Vesturlands
þingflokkur jafnaðarmanna
Hreyfingin
Hreyfingin
firðinga
Vesturlands
Samtök
Samtök frjálslyndra og vinstri manna
R
Hreyfingin
Samtök
Samtök
Samtök
Samtök
Akureyrar
Samtök
Reykvíkinga
Framfaraflokkur
flokkur
ram
flokkur
Samtök
Samtök
varaþingmaður
R
Reykvíkinga
H
varaþingmaður
Eyfirðinga
Eyfirðinga
Siglfirðinga
G
Akureyrar
Reykvíkinga
Eyfirðinga
Reyknesinga
VesturSkaftfellinga
varaþingmaður
Bandalag
Gull
Suðurlands
Flokkur
Flokkur fólksins
Flokkur
Flokkur fólksins
Samtök
Samtök
Samtök
þingflokkur
Mýramanna
Reyknesinga
Ísafjarðar
varaþingmaður
Samtök
Samtök frjálslyndra og vinstri manna

Samtök
Samtök
Samtök frjálslyndra og vinstri manna
flokkur
NorðurÞingeyinga
Alþýðu
Vinstrihreyf­ingin
Austurlands
ram
flokkur
Nor
Samtök
Samtök frjálslyndra og vinstri manna
varaþingmaður
Samtök
Samtök frjálslyndra og vinstri manna
Flokkur
Flokkur fólksins
Vesturlands
Samtök
Samtök
Rangæinga
Samtök
Samtök
Kötlum
H

In [87]:
member_party

{'Adda María Jóhannsdóttir': {'Samfylkingin'},
 'Adda Bára Sigfúsdóttir': {'Alþýðubandalagið'},
 'Adolf H Berndsen': {'Sjálfstæðisflokkurinn'},
 'Aðalheiður Bjarnfreðsdóttir': {'Borgaraflokkurinn'},
 'Aðalsteinn Haukur Sverrisson': set(),
 'Albert Guðmundsson': {'Sjálfstæðisflokkurinn'},
 'Albert Sigurður': {'Borgaraflokkurinn', 'Sjálfstæðisflokkurinn'},
 'Albertína Friðbjörg Elíasdóttir': {'Samfylkingin'},
 'Alex B Stefánsson': {'Framsóknarflokkur'},
 'Alexander Stefánsson': {'Framsóknarflokkurinn'},
 'Alfreð Gíslason': {'Sjálfstæðisflokkurinn'},
 'Alma Lísa Jóhannsdóttir': {'VG'},
 'Amal Tamimi': {'Samfylkingin'},
 'Andrés Eyjólfsson': {'Framsóknarflokkurinn'},
 'Andrés Ingi Jónsson': {'Píratar', 'VG'},
 'Andri Þór Sturluson': {'Píratar'},
 'Angantýr Guðjónsson': {'Sjálfstæðisflokkurinn'},
 'Anna Kolbrún Árnadóttir': {'Miðflokkurinn'},
 'Anna Ólafsdóttir Björnsson': {'Samtök um kvennalista'},
 'Anna María Elíasdóttir': {'Framsóknarflokkur'},
 'Anna Margrét Guðjónsdóttir': {'Samfylkin

In [72]:
alþingi[19]

'    Anna Ólafsdóttir Björnsson. Alþingismaður Reyknesinga 1989–1995 (Samtök um kvennalista).\n'

In [73]:
re.findall("\(((?:\w|\s)*)\)\.", alþingi[19])

['Samtök um kvennalista']

In [74]:
for sentence in alþingi[:50]:
    
    # ML methods
    member = get_names(sentence)
    party = get_party(sentence)
    
    # RE methods
    # search = re.search("^(.*)\. ", sentence)
    # member = search[0]
    party = set(re.findall("\((\w*)\)\.", sentence))
    
    print(member, party)

{'Adda María Jóhannsdóttir'} {'Samfylkingin'}
{'Adda Bára Sigfúsdóttir'} {'Alþýðubandalag'}
{'Adolf H Berndsen'} {'Sjálfstæðisflokkur'}
{'Aðalheiður Bjarnfreðsdóttir'} {'Borgaraflokkur'}
{'Aðalsteinn Haukur Sverrisson'} set()
{'Albert Guðmundsson', 'Albert Sigurður'} {'Borgaraflokkur'}
{'Albert Guðmundsson'} {'Sjálfstæðisflokkur'}
{'Albertína Friðbjörg Elíasdóttir'} {'Samfylkingin'}
{'Alex B Stefánsson'} {'Framsóknarflokkur'}
{'Alexander Stefánsson'} {'Framsóknarflokkur'}
{'Alfreð Gíslason'} {'Alþýðubandalag'}
Reyk
inga
{'Alfreð Gíslason'} {'Sjálfstæðisflokkur'}
{'Alma Lísa Jóhannsdóttir'} set()
{'Amal Tamimi'} {'Samfylkingin'}
{'Andrés Eyjólfsson'} {'Framsóknarflokkur'}
{'Andrés Ingi Jónsson'} set()
{'Andri Þór Sturluson'} {'Píratar'}
{'Angantýr Guðjónsson'} {'Sjálfstæðisflokkur'}
{'Anna Kolbrún Árnadóttir'} {'Miðflokkurinn'}
Samtök
{'Anna Ólafsdóttir Björnsson'} set()
{'Anna María Elíasdóttir'} {'Framsóknarflokkur'}
{'Anna Margrét Guðjónsdóttir'} {'Samfylkingin'}
{'Anna Kristín Gunna

KeyboardInterrupt: 

In [75]:
flokkar = set()
for it in member_party.values():
    for i in it:
        flokkar.add(i)

In [76]:
flokkar

{'Alþýðubandalagið',
 'Alþýðuflokkurinn',
 'Björt framtíð',
 'Borgaraflokkur',
 'Borgaraflokkurinn',
 'Borgarahreyfingin',
 'Bændaflokkurinn',
 'Framfaraflokkurinn',
 'Framsóknarflokkur',
 'Framsóknarflokkurinn',
 'Frjálslyndi',
 'Heimastjórnarflokkurinn',
 'Kommúnistaflokkurinn',
 'Kvennalistinn',
 'Landvarnarflokkurinn',
 'Miðflokkurinn',
 'Píratar',
 'Sambandsflokkurinn',
 'Sameiningarflokkur',
 'Samfylkingin',
 'Samtök um jafnrétti og félagshyggju',
 'Samtök um kvennalista',
 'Sjálfstæðisflokkurinn',
 'Sparnaðarbandalagið',
 'Sósíalistaflokkurinn',
 'Utanflokkabandalagið',
 'VG',
 'Viðreisn',
 'Íhaldsflokkurinn',
 'Þjóðræðisflokkurinn',
 'Þjóðvaki',
 'Þjóðvarnarflokkurinn'}

In [77]:
for key, value in member_party.items():
    if not value:
        print(key)

Aðalsteinn Haukur Sverrisson
Anna Karen Svövudóttir
Arnar Þór Jónsson
Arndís Anna Kristínardóttir Gunnarsdóttir
Arnljótur Ólafsson
Ágúst Bjarni Garðarsson
Ágústa Guðmundsdóttir
Árni Böðvarsson
Árni Einarsson
Árni Helgason
Ásgeir Einarsson
Ásta Guðrún Helgadóttir
Ásta B Þorsteinsdóttir
Ásthildur Lóa Þórsdóttir
Baldvin Jónsson
Benedikt Kristjánsson
Benedikt Þórðarson
Benóný Arnórsson
Berglind Ósk Guðmundsdóttir
Berglind Harpa Svavarsdóttir
Bergur Thorberg
Bjarni Guðnason
Bjarni Johnsen
Bjarni Thorsteinson
Björgvin Jóhannesson
Björn Blöndal
Björn Halldórsson
Björn Pétursson
Björn Þórðarson
Bogi Th Melsteð
Brynhildur Björnsdóttir
Brynja Dan Gunnarsdóttir
Brynjólfur Benedictsen
Brynjólfur Jónsson
Daði Már Kristófersson
Dagbjört Hákonardóttir
Daníel Thorlacius
Davíð Guðmundsson
Diljá Mist Einarsdóttir
Eggert Briem
Eggert Gunnarsson
Egill Egilsson
Einar Ásmundsson
Einar Gíslason
Einar B Guðmundsson
Einar Thorlacius
Eiríkur Gíslason
Friðrik Eggerz
Eiríkur Ó Kúld
Elín Anna Gísladóttir
Erlendur 

In [78]:
# Ath að einræða flokka, greinir/ekki greinir
flokkasia = ['Alþýðubandalag',
 'Alþýðubandalagið',
 'Alþýðuflokkur',
 'Alþýðuflokkurinn',
 'Borgaraflokkur',
 'Borgaraflokkurinn',
 'Borgarahreyfingin',
 'Bændaflokkur',
 'Bændaflokkurinn',
 'Framfaraflokkurinn',
 'Framsóknar',
 'Framsóknarflokkur',
 'Framsóknarflokkurinn',
 'Frjálslyndi',
 'Heimastjórnarflokkurinn',
 'Kommúnistaflokkurinn',
 'Landvarnarflokkurinn',
 'Miðflokkurinn',
 'Píratar',
 'Sambandsflokkurinn',
 'Sameiningarflokkur',
 'Samfylkingin',
 'Sjálfstæðisflokkur',
 'Sjálfstæðisflokkurinn',
 'Sparnaðarbandalagið',
 'Sósíalistaflokkur',
 'Sósíalistaflokkurinn',
 'Utanflokkabandalagið',
 'Vinstrihreyfingin',
 'Viðreisn',
 'Íhaldsflokkurinn',
 'Þjóðræðisflokkurinn',
 'Þjóðvaki',
 'Þjóðvarnarflokkurinn',]

In [79]:
Vandkvæði = {'vennalistinn',
'reisn',
'ramsóknarflokkur',
'Vinstrihreyf\xadingin',
'Hreyfingin',
'Björt',
'Alþýðu',
'þingflokkur'}

In [80]:
data_train.loc[jalist[5]]

title                          Vilja sóðaskatt og allir verði vegan
author                                       Kristín Sigurðardóttir
timestamp                                       2017-09-10 17:38:45
text              Sóðaskattur, framleiðsla á áburði með ánamöðku...
text_tokenized    Sóðaskattur , framleiðsla á áburði með ánamöðk...
Name: 620326, dtype: object

In [81]:
text = data_train.loc[543888]["text"]
names = get_names(text)
names_nf = nefnifall(names)
for n in names:
    if n in member_party:
        print(n, member_party[n])
print()
print(names, names_nf)

Guðlaugur Þór Þórðarson {'Sjálfstæðisflokkurinn'}

{'Guðlaugur Þór Þórðarson', 'Guðlaugur Þór'} ['Guðlaugur Þórr Þórðarson', 'Guðlaugur Þórr']


Ath f. frétt 543888, nefnifall bætir við r-i aftan á 'Þór'!

In [82]:
with open("Current_alþingismenn.txt", "r") as f:
    current_alþingi = f.readlines()

current_member_party = dict()
for line in current_alþingi:
    line = line.strip()
    line = line.split("   ")
    current_member_party[line[0]] = line[1]
    

In [83]:
current_member_party

{'Andrés Ingi Jónsson': 'Píratar',
 'Arndís Anna Kristínardóttir Gunnarsdóttir': 'Píratar',
 'Ágúst Bjarni Garðarsson': 'Framsóknarflokkur',
 'Áslaug Arna Sigurbjörnsdóttir': 'Sjálfstæðisflokkur',
 'Ásmundur Einar Daðason': 'Framsóknarflokkur',
 'Ásmundur Friðriksson': 'Sjálfstæðisflokkur',
 'Ásthildur Lóa Þórsdóttir': 'Flokkur fólksins',
 'Berglind Ósk Guðmundsdóttir': 'Sjálfstæðisflokkur',
 'Bergþór Ólason': 'Miðflokkurinn',
 'Birgir Ármannsson': 'Sjálfstæðisflokkur',
 'Birgir Þórarinsson': 'Sjálfstæðisflokkur',
 'Bjarkey Olsen Gunnarsdóttir': 'Vinstrihreyfingin - grænt framboð',
 'Bjarni Benediktsson': 'Sjálfstæðisflokkur',
 'Bjarni Jónsson': 'Vinstrihreyfingin - grænt framboð',
 'Björn Leví Gunnarsson': 'Píratar',
 'Bryndís Haraldsdóttir': 'Sjálfstæðisflokkur',
 'Diljá Mist Einarsdóttir': 'Sjálfstæðisflokkur',
 'Eyjólfur Ármannsson': 'Flokkur fólksins',
 'Gísli Rafn Ólafsson': 'Píratar',
 'Guðbrandur Einarsson': 'Viðreisn',
 'Guðlaugur Þór Þórðarson': 'Sjálfstæðisflokkur',
 'Guðmun

In [84]:
current_flokkar = set()
for it in current_member_party.values():
    current_flokkar.add(it)

In [85]:
current_flokkar

{'Flokkur fólksins',
 'Framsóknarflokkur',
 'Miðflokkurinn',
 'Píratar',
 'Samfylkingin',
 'Sjálfstæðisflokkur',
 'Vinstrihreyfingin - grænt framboð',
 'Viðreisn'}

In [121]:
# Detect names/parties mentioned article, update count-dictionaries for each
# Before: text = article text, 
#         member_party_dict = dictionary in the format member : party
#         member_hits, party_hits: counter dictionaries in the format of
#              member of alþingi : number of "hits" or articles mentioning member
#              political party : num of articles mentioning party
# After: Returns updated counters

def count_hits(text, member_party_dict, member_hits, party_hits):
    names = get_names(text)
    for nafn in names:
        print(nafn)
        print(nefnifall_stk(nafn))
        if (nefnifall_stk(nafn) in member_party_dict or nafn in member_party_dict):
            flokkur = member_party_dict[nafn] # this should already be declined in nf.
            print(flokkur)
            if nafn in member_hits:
                member_hits[nafn] += 1
            else: member_hits[nafn] = 1
            if flokkur in party_hits:
                party_hits[flokkur] += 1
            else: party_hits[flokkur] = 1    
    return member_hits, party_hits


# Initialize the hit counters
member_hits = dict()
party_hits = dict()

# Test on a the "Þórr" article
text_tmp = data_train.loc[543888]["text"]
member_hits, party_hits = count_hits(text_tmp, current_member_party, member_hits, party_hits)

print(member_hits)
print(party_hits)

Guðlaugur Þór Þórðarson
Guðlaugur Þórr Þórðarson
Sjálfstæðisflokkur
Guðlaugur Þór
Guðlaugur Þórr
{'Guðlaugur Þór Þórðarson': 2}
{'Sjálfstæðisflokkur': 2}


In [122]:
# Start new counters for current alþingismenn/flokkar
current_member_hits = dict()
current_party_hits = dict()

# Count current members and parties mentioned in a list environmentally-centered news (jálist)


In [123]:
for i in jalist:
    a = corpus_lsi_200[data_train.index.get_loc(i)]
    sim = matutils.cossim(a,t)
    print(sim, data_train.loc[i]["title"])

NameError: name 'corpus_lsi_200' is not defined